<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4b9bb4cf175ec090d1e5b74ffdac32293aa52903a7501136991d506002294b71
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-09 14:09:07
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.40 C
-------------------
Today PnL: 49.73 K (0.36%)
Current PnL: -18.85 L (-12.68%)
CY Booked + Current PnL: -7.82 L (-5.26%)
-------------------
Total profit:  1.72 L
Total loss:  -20.57 L
-------------------
Total Booked + Current PnL: 19.16 L (15.74%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.86%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.50 L (55.92%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.86,59.0,H-LC,10.78,172040.0,15863.0,10236.0,0.43,10.16,5.95,16.71,22.0,1.55,1.33,29.37,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.76,63.0,M-SC,5.88,88608.0,-12169.0,12263.0,0.62,-12.08,13.84,0.09,245.0,-0.99,0.69,16.41,OX40N,NTT,DURABLES
62,SBIN,863.00,-13.25,48.0,M-LC,4.29,214213.0,12734.0,14481.0,-0.06,6.32,6.76,13.51,60.0,0.88,1.66,19.71,XY25,NTT,BANKS
41,ITC,452.00,-37.92,51.0,H-LC,3.21,200686.0,548.0,20350.0,0.70,0.27,10.14,10.44,4.0,0.03,1.55,5.85,X40,NTT,FMCG
49,NATIONALUM,247.44,-45.32,65.0,H-MC,1.72,109779.0,9275.0,21363.0,-1.63,9.23,19.46,30.49,79.0,0.43,0.85,44.62,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.14,61.0,M-MC,8.02,223343.0,-1619.0,169272.0,-1.33,-0.72,75.79,74.53,192.0,-0.01,1.73,31.30,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.67,40.0,H-SC,6.49,192044.0,1038.0,56653.0,0.04,0.54,29.50,30.21,157.0,0.02,1.48,21.54,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-21.80,36.0,H-MC,2.08,134470.0,-1318.0,44066.0,0.31,-0.97,32.77,31.48,107.0,-0.03,1.04,11.03,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.23,61.0,H-MC,1.80,247354.0,-377.0,75443.0,0.41,-0.15,30.50,30.30,92.0,-0.00,1.91,13.12,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-21.04,40.0,H-MC,3.72,135775.0,-358.0,31486.0,0.10,-0.26,23.19,22.86,91.0,-0.01,1.05,12.32,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,JIOFIN,387.00,-2.61,38.0,H-LC,13.87,215460.0,-1946.0,55438.0,-1.06,-0.90,25.73,24.61,48.0,-0.04,1.67,53.43,XY24,BTT,FINANCE
31,HCLTECH,1943.91,-1.94,38.0,H-LC,10.17,223960.0,-17964.0,81230.0,1.67,-7.43,36.27,26.15,8.0,-0.22,1.73,5.83,X40,ATH,IT
83,VBL,671.64,-15.83,39.0,H-LC,6.66,302922.0,-12920.0,125591.0,0.16,-4.09,41.46,35.67,5.0,-0.10,2.34,9.20,X40N,ATH,FMCG
74,TCS,4998.00,-28.04,42.0,H-LC,14.19,286324.0,-59632.0,183476.0,0.89,-17.24,64.08,35.80,1.0,-0.33,2.21,1.43,X40,BTT,IT
80,UNITDSPR,1644.00,-14.66,44.0,H-LC,8.35,171785.0,-9974.0,45231.0,0.42,-5.49,26.33,19.39,86.0,-0.22,1.33,2.21,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-5.78,52.0,H-LC,1.59,233685.0,-12466.0,91675.0,2.04,-5.06,39.23,32.18,16.0,-0.14,1.81,29.15,X200,ATH,IT
50,NESTLEIND,1377.00,-8.86,58.0,H-LC,2.52,280496.0,15070.0,41710.0,0.93,5.68,14.87,21.40,11.0,0.36,2.17,13.42,XY25,NTT,FMCG
41,ITC,452.00,-37.92,51.0,H-LC,3.21,200686.0,548.0,20350.0,0.70,0.27,10.14,10.44,4.0,0.03,1.55,5.85,X40,NTT,FMCG
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.81,212830.0,8330.0,33095.0,0.80,4.07,15.55,20.25,10.0,0.25,1.65,13.16,X40N,BTT,PHARMA
83,VBL,671.64,-15.83,39.0,H-LC,6.66,302922.0,-12920.0,125591.0,0.16,-4.09,41.46,35.67,5.0,-0.10,2.34,9.20,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,69.0,H-SC,12.87,128250.0,-14229.0,33755.0,7.54,-9.99,26.32,13.70,163.0,-0.42,0.99,51.37,OX40N,NTT,BANKS
81,VAIBHAVGBL,521.0,57.02,66.0,H-SC,11.02,104681.0,-47310.0,121430.0,5.64,-31.13,116.00,48.77,125.0,-0.39,0.81,21.84,XR,NTT,JEWELLERY
39,INFY,2275.0,-17.80,53.0,H-LC,10.55,319969.0,7081.0,164592.0,4.84,2.26,51.44,54.87,3.0,0.04,2.47,9.06,X40,BTT,IT
82,VALIANTORG,838.0,-338.30,66.0,H-SC,8.91,101296.0,-31990.0,116582.0,4.56,-24.00,115.09,63.47,139.0,-0.27,0.78,69.80,XR,NTT,CHEMICALS
75,TITAGARH,1548.0,-6.48,58.0,H-SC,7.85,153492.0,-38523.0,117406.0,3.80,-20.06,76.49,41.08,158.0,-0.33,1.19,30.07,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3737.84,64.0,M-SC,7.79,119699.0,-27669.0,79480.0,-5.23,-18.78,66.40,35.16,236.0,-0.35,0.93,22.26,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-39.64,60.0,L-SC,40.45,90720.0,-22829.0,62878.0,-3.20,-20.10,69.31,35.27,268.0,-0.36,0.70,122.04,XR,NTT,HOTELS
68,STARHEALTH,761.00,7.79,50.0,H-SC,3.72,160373.0,-39843.0,115869.0,-2.17,-19.90,72.25,37.97,144.0,-0.34,1.24,28.77,XY24,NTT,INSURANCE
67,SONACOMS,1006.00,-37.48,46.0,M-SC,12.33,77805.0,-23354.0,98244.0,-2.15,-23.09,126.27,74.03,202.0,-0.24,0.60,9.71,AR,BTT,AUTO
49,NATIONALUM,247.44,-45.32,65.0,H-MC,1.72,109779.0,9275.0,21363.0,-1.63,9.23,19.46,30.49,79.0,0.43,0.85,44.62,MH,ATH,METALS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,69.0,H-SC,12.87,128250.0,-14229.0,33755.0,7.54,-9.99,26.32,13.70,163.0,-0.42,0.99,51.37,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.76,63.0,M-SC,5.88,88608.0,-12169.0,12263.0,0.62,-12.08,13.84,0.09,245.0,-0.99,0.69,16.41,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,64.0,H-SC,2.58,224892.0,-44775.0,81119.0,2.70,-16.60,36.07,13.47,138.0,-0.55,1.74,13.75,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.29,49.0,M-SC,3.24,140666.0,-33893.0,33929.0,-0.63,-19.42,24.12,0.02,221.0,-1.00,1.09,22.35,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.41,114120.0,-29683.0,56432.0,-0.70,-20.64,49.45,18.60,149.0,-0.53,0.88,25.93,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-28.04,42.0,H-LC,14.19,286324.0,-59632.0,183476.0,0.89,-17.24,64.08,35.80,1.0,-0.33,2.21,1.43,X40,BTT,IT
39,INFY,2275.00,-17.80,53.0,H-LC,10.55,319969.0,7081.0,164592.0,4.84,2.26,51.44,54.87,3.0,0.04,2.47,9.06,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,3.21,200686.0,548.0,20350.0,0.70,0.27,10.14,10.44,4.0,0.03,1.55,5.85,X40,NTT,FMCG
83,VBL,671.64,-15.83,39.0,H-LC,6.66,302922.0,-12920.0,125591.0,0.16,-4.09,41.46,35.67,5.0,-0.10,2.34,9.20,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,46.0,H-LC,12.75,246485.0,-15134.0,134359.0,0.52,-5.78,54.51,45.57,7.0,-0.11,1.91,4.88,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.17,53.0,H-MC,3.36,188500.0,23830.0,44147.0,0.04,14.47,23.42,41.28,89.0,0.54,1.46,23.61,X40N,ATH,AC
4,ANGELONE,3033.00,8.67,40.0,H-SC,6.49,192044.0,1038.0,56653.0,0.04,0.54,29.50,30.21,157.0,0.02,1.48,21.54,X40N,NTT,FINANCE
84,VOLTAS,1918.49,1.02,59.0,H-MC,3.48,211815.0,20073.0,75957.0,-0.50,10.47,35.86,50.08,99.0,0.26,1.64,17.98,XY25,ATH,AC
24,DABUR,735.00,-2.58,61.0,H-MC,5.15,212121.0,15093.0,74539.0,-0.42,7.66,35.14,45.49,102.0,0.20,1.64,21.70,XY24,BTT,FMCG
18,CAMS,5211.76,-5.38,50.0,H-SC,5.28,107668.0,5664.0,38265.0,0.34,5.55,35.54,43.06,122.0,0.15,0.83,23.62,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-45.32,65.0,H-MC,1.72,109779.0,9275.0,21363.0,-1.63,9.23,19.46,30.49,79.0,0.43,0.85,44.62,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-38.77,65.0,M-SC,3.08,103455.0,11956.0,66791.0,1.20,13.07,64.56,86.07,223.0,0.18,0.80,50.24,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.28,48.0,H-SC,12.10,136072.0,12736.0,121022.0,-0.28,10.33,88.94,108.45,119.0,0.11,1.05,34.99,AR,ATH,MISC
86,WIPRO,420.00,-11.98,51.0,M-LC,6.75,154473.0,3528.0,105922.0,2.73,2.34,68.57,72.51,53.0,0.03,1.19,8.50,XR,NTT,IT
12,BANKINDIA,190.00,-32.92,48.0,H-MC,6.58,173394.0,-6414.0,118827.0,-0.41,-3.57,68.53,62.52,88.0,-0.05,1.34,29.30,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6722.22,52.0,L-SC,6.20,74876.0,-18934.0,95549.0,1.36,-20.18,127.61,81.67,269.0,-0.20,0.58,45.00,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-6.52,40.0,H-SC,14.98,86993.0,-13970.0,108498.0,-0.61,-13.84,124.72,93.63,148.0,-0.13,0.67,29.57,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,60.0,L-SC,40.45,90720.0,-22829.0,62878.0,-3.20,-20.10,69.31,35.27,268.0,-0.36,0.70,122.04,XR,NTT,HOTELS
48,MASFIN,398.61,-15.99,60.0,H-SC,12.95,95415.0,-2565.0,24169.0,2.48,-2.62,25.33,22.05,152.0,-0.11,0.74,38.69,XR,ATH,FINANCE
82,VALIANTORG,838.00,-338.30,66.0,H-SC,8.91,101296.0,-31990.0,116582.0,4.56,-24.00,115.09,63.47,139.0,-0.27,0.78,69.80,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,60.0,L-SC,40.45,90720.0,-22829.0,62878.0,-3.20,-20.10,69.31,35.27,268.0,-0.36,0.70,122.04,XR,NTT,HOTELS
8,ATULAUTO,844.0,3737.84,64.0,M-SC,7.79,119699.0,-27669.0,79480.0,-5.23,-18.78,66.40,35.16,236.0,-0.35,0.93,22.26,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-171.59,63.0,M-SC,4.60,81876.0,-20334.0,113726.0,-0.91,-19.89,138.90,91.37,208.0,-0.18,0.63,32.91,XY25,NTT,FINANCE
82,VALIANTORG,838.0,-338.30,66.0,H-SC,8.91,101296.0,-31990.0,116582.0,4.56,-24.00,115.09,63.47,139.0,-0.27,0.78,69.80,XR,NTT,CHEMICALS
55,RELAXO,1176.0,-31.94,59.0,H-SC,3.39,63168.0,-52370.0,87361.0,1.01,-45.33,138.30,30.28,136.0,-0.60,0.49,24.19,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.30
1,25,44.50
2,50,75.07


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.91
LC    35.32
MC    23.79
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.73
X40      16.03
X40N     13.16
XY25     10.32
XR        9.55
AR        8.31
OX40N     7.77
X200      1.81
MH        1.78
X5K       1.45
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.78
H-SC    24.99
H-MC    20.69
M-SC    14.24
M-LC     7.43
M-MC     2.77
L-SC     1.68
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.78,-17.89,77.57
FMCG,12.35,-3.88,39.10
FINANCE,9.09,-18.92,61.68
BANKS,7.68,-14.89,63.67
PAINTS,6.21,-14.98,40.72
MISC,6.02,-17.80,85.32
ELECTRICAL,5.88,-8.41,68.16
AC,3.85,2.65,30.31
AUTO,3.54,-14.81,66.58


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2751711.0,22
AR,1196621.0,9
XR,1058475.0,13
X40,892010.0,10
X40N,574365.0,10
OX40N,453213.0,10
XY25,417369.0,7
SR,199422.0,2
X5K,129631.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2791572.0,28
M-SC,1405159.0,17
H-LC,1402672.0,15
H-MC,1169407.0,15
M-LC,417590.0,5
M-MC,319291.0,2
L-SC,243691.0,3
L-MC,60052.0,1
L-LC,40428.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,924263.0,6
M-SC,XY24,818173.0,7
H-SC,AR,661859.0,5
H-LC,X40,601217.0,5
H-SC,XR,499326.0,6
H-MC,XY24,466990.0,4
H-LC,AR,374103.0,2
M-MC,XY24,319291.0,2
H-SC,X40N,250430.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
